In [44]:
!pip install scikit-multilearn

import re
import os
import tqdm
import nltk
import pickle
import sqlite3
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import xgboost as xgb
import tensorflow as tf
from sklearn import metrics
from tensorflow import keras
from nltk.corpus import words
from datetime import datetime
from bs4 import BeautifulSoup
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from itertools import combinations
from keras.models import load_model
from keras.models import Sequential
from tensorflow.keras import layers
from nltk.stem import SnowballStemmer
from sklearn.pipeline import Pipeline
from nltk.tokenize import sent_tokenize
from keras.preprocessing import sequence
from scipy.sparse import coo_matrix, hstack
from tensorflow.keras.utils import plot_model
from keras.layers.embeddings import Embedding
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import RandomizedSearchCV
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score,precision_score,recall_score,hamming_loss
from keras.layers import Conv1D, Conv2D, Dense, Dropout, Flatten, LSTM, GlobalMaxPooling1D, MaxPooling2D, Activation, BatchNormalization

%matplotlib inline
nltk.download('punkt')
nltk.download('wordnet')
warnings.filterwarnings("ignore")
stemmer = SnowballStemmer('english')

%autosave 120

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bhanu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bhanu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Autosaving every 120 seconds


In [3]:
data = pd.read_csv("data_with_all_tags.csv")
data.head(2)

,title,plot_synopsis,tags,split,synopsis_source,cnt_dup,tag_count,synopsis_count,synopsis_sent_count,CleanedSynopsis,...,sentimental,storytelling,stupid,suicidal,suspenseful,thought-provoking,tragedy,violence,western,whimsical
0,$,"Set in Hamburg, West Germany, several criminal...",murder,test,imdb,1,1,648,26,set hamburg west germani sever crimin take adv...,...,0,0,0,0,0,0,0,0,0,0
1,$windle,A 6th grader named Griffin Bing decides to gat...,flashback,train,wikipedia,1,1,353,14,grader name griffin bing decid gather entir gr...,...,0,0,0,0,0,0,0,0,0,0


In [6]:
vectorizer = CountVectorizer(preprocessor=lambda x: x, tokenizer = lambda x: str(x).split(", ") )
tag_vect = vectorizer.fit_transform(data["tags"])

In [7]:
tags = vectorizer.get_feature_names()
freqs = tag_vect.sum(axis=0).A1
result = list(zip(tags, freqs))

print((result))

[('absurd', 259), ('action', 605), ('adult comedy', 124), ('allegory', 123), ('alternate history', 98), ('alternate reality', 198), ('anti war', 108), ('atmospheric', 385), ('autobiographical', 43), ('avant garde', 213), ('blaxploitation', 73), ('bleak', 210), ('boring', 519), ('brainwashing', 103), ('christian film', 39), ('claustrophobic', 80), ('clever', 86), ('comedy', 1795), ('comic', 110), ('cruelty', 422), ('cult', 2531), ('cute', 190), ('dark', 387), ('depressing', 196), ('dramatic', 402), ('entertaining', 726), ('fantasy', 492), ('feel-good', 74), ('flashback', 2801), ('good versus evil', 793), ('gothic', 400), ('grindhouse film', 61), ('haunting', 132), ('historical', 266), ('historical fiction', 118), ('home movie', 148), ('horror', 455), ('humor', 803), ('insanity', 585), ('inspiring', 116), ('intrigue', 154), ('magical realism', 51), ('melodrama', 423), ('murder', 5375), ('mystery', 500), ('neo noir', 700), ('non fiction', 32), ('paranormal', 501), ('philosophical', 222), 

In [8]:
tag_counts = pd.DataFrame(result,columns=['tag','tag_counts'])
tag_counts.head()

,tag,tag_counts
0,absurd,259
1,action,605
2,adult comedy,124
3,allegory,123
4,alternate history,98


In [9]:
tag_counts_sorted = tag_counts.sort_values(['tag_counts'], ascending=False)

In [10]:
tag_counts = tag_counts_sorted['tag'][:30]
tag_counts

43              murder
68            violence
28           flashback
57            romantic
20                cult
56             revenge
52         psychedelic
17              comedy
65         suspenseful
37               humor
29    good versus evil
59              satire
25        entertaining
45            neo noir
58              sadist
1               action
38            insanity
67             tragedy
12              boring
47          paranormal
44             mystery
26             fantasy
36              horror
42           melodrama
19             cruelty
24            dramatic
30              gothic
22                dark
7          atmospheric
62        storytelling
Name: tag, dtype: object

In [11]:
tags_30 = list(tag_counts)

In [12]:
data['tags']

0                                                   murder
1                                                flashback
2                  suspenseful, neo noir, murder, violence
3                                           cult, violence
4        murder, anti war, violence, flashback, tragedy...
                               ...                        
13752                                   paranormal, murder
13753                                             romantic
13754                                     autobiographical
13755    cruelty, murder, cult, violence, flashback, ps...
13756                                               murder
Name: tags, Length: 13757, dtype: object

In [13]:
#Deleting Every Tags other than TOP 30 Tags

for le in tqdm(range(data.shape[0])):
    a = []
    b = data['tags'][le].split(",")
    c = ''
    
    for i in range(len(b)):        
        for j in tags_30:
            temp = b[i].strip()  #Stripping the white-spaces present(if any) around the tag
            
            if temp == j:        #Comparing with the TOP 30 tags
                a.append(j)
        
    if a:
        data['tags'][le] = ", ".join(a)
    else:
         data['tags'][le] = np.nan        

100%|███████████████████████████████████████████████████████████████████████████| 13757/13757 [00:17<00:00, 804.22it/s]


In [14]:
#Deleting the rows that have NaN Values
data = data.dropna() 

In [15]:
data['tags']

0                                                   murder
1                                                flashback
2                  suspenseful, neo noir, murder, violence
3                                           cult, violence
4                     murder, violence, flashback, tragedy
                               ...                        
13751                            comedy, sadist, flashback
13752                                   paranormal, murder
13753                                             romantic
13755    cruelty, murder, cult, violence, flashback, ps...
13756                                               murder
Name: tags, Length: 13010, dtype: object

In [16]:
vectorizer = CountVectorizer(preprocessor=lambda x: x, tokenizer = lambda x: str(x).split(", "))
tag_vect = vectorizer.fit_transform(data["tags"])

In [17]:
tags = vectorizer.get_feature_names()
freqs = tag_vect.sum(axis=0).A1
result = list(zip(tags, freqs))

print((result))

[('action', 605), ('atmospheric', 385), ('boring', 519), ('comedy', 1795), ('cruelty', 422), ('cult', 2531), ('dark', 387), ('dramatic', 402), ('entertaining', 726), ('fantasy', 492), ('flashback', 2801), ('good versus evil', 793), ('gothic', 400), ('horror', 455), ('humor', 803), ('insanity', 585), ('melodrama', 423), ('murder', 5375), ('mystery', 500), ('neo noir', 700), ('paranormal', 501), ('psychedelic', 1800), ('revenge', 2313), ('romantic', 2729), ('sadist', 618), ('satire', 780), ('storytelling', 346), ('suspenseful', 1026), ('tragedy', 536), ('violence', 4138)]


In [18]:
len(result)

30

In [19]:
conn = sqlite3.connect('data30.db')
data.to_sql('data30', conn, if_exists='replace', index=False)
train = pd.read_sql("Select * From data30 where split = 'train' OR split='val'",conn)
test =  pd.read_sql("Select * From data30 where split = 'test'",conn)
conn.close()

In [20]:
X_train = train["CleanedSynopsis"]
y_train= train["tags"]

X_test = test["CleanedSynopsis"]
y_test= test["tags"]

In [21]:
cnt_vectorizer = CountVectorizer(binary='true', max_features = 30).fit(y_train)
y_train_multilabel = cnt_vectorizer.transform(y_train)
y_test_multilabel = cnt_vectorizer.transform(y_test)

In [22]:
print("Dimensions of data Y_train_multilabel:",y_train_multilabel.shape,"Y_test_multilabel:",y_test_multilabel.shape)

Dimensions of data Y_train_multilabel: (10375, 30) Y_test_multilabel: (2635, 30)


# Word2Vec

In [23]:
X_train_new = []

for i in tqdm(range(len(list(X_train)))):
    X_train_new.append(X_train[i].split(" "))

100%|██████████████████████████████████████████████████████████████████████████| 10375/10375 [00:02<00:00, 5118.45it/s]


In [25]:
with open('pickled glove 6b 300d.pkl', 'rb') as f:
    new_model = pickle.load(f)
    words =  set(new_model.keys())

In [26]:
X_train_multilabel = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(X_train.values): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split():
        if word in words:
            vector += new_model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    X_train_multilabel.append(vector)

100%|███████████████████████████████████████████████████████████████████████████| 10375/10375 [00:46<00:00, 223.80it/s]


In [27]:
X_test_multilabel = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(X_test.values): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split():
        if word in words:
            vector += new_model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    X_test_multilabel.append(vector)

100%|█████████████████████████████████████████████████████████████████████████████| 2635/2635 [00:12<00:00, 214.52it/s]


<h1>OneVsRestClassifier + SGDClassifier with LOG Loss :</h1>

In [28]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced',
                                            loss='log'))

In [29]:
prediction1 = clf.predict(X_test_multilabel)

precision1 = precision_score(y_test_multilabel, prediction1, average='micro')

recall1 = recall_score(y_test_multilabel, prediction1, average='micro')

f1_score1 = 2*((precision1 * recall1)/(precision1 + recall1))

print("precision1: {:.4f}, recall1: {:.4f}, F1-measure: {:.4f}".format(precision1, recall1, f1_score1))

precision1: 0.1991, recall1: 0.6642, F1-measure: 0.3064


In [30]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction1[k])[0],"\n")

Movie:  Dilwale Dulhania Le Jayenge
Actual genre:  romantic
Predicted tag:  ['melodrama' 'romantic' 'tragedy'] 

Movie:  Scooby-Doo Meets the Boo Brothers
Actual genre:  psychedelic, horror
Predicted tag:  ['boring' 'evil' 'fantasy' 'good' 'gothic' 'horror' 'murder' 'mystery'
 'paranormal' 'revenge' 'versus'] 

Movie:  The Bodyguard
Actual genre:  murder, violence, cult, action, romantic, melodrama, suspenseful
Predicted tag:  ['boring' 'comedy' 'cruelty' 'dramatic' 'flashback' 'murder' 'mystery'
 'neo' 'noir' 'revenge' 'sadist' 'suspenseful' 'violence'] 

Movie:  Good Night, and Good Luck.
Actual genre:  suspenseful, flashback
Predicted tag:  ['boring' 'dramatic' 'entertaining' 'humor' 'melodrama' 'mystery'
 'romantic' 'satire'] 

Movie:  Alien
Actual genre:  dark, boring, gothic, murder, mystery, cult, violence, horror, atmospheric, action, suspenseful
Predicted tag:  ['boring' 'cult' 'entertaining' 'evil' 'good' 'gothic' 'horror' 'humor'
 'insanity' 'melodrama' 'mystery' 'paranormal

<h2> OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [31]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced'))

In [32]:
prediction2 = clf.predict(X_test_multilabel)

precision2 = precision_score(y_test_multilabel, prediction2, average='micro')

recall2 = recall_score(y_test_multilabel, prediction2, average='micro')

f1_score2 = 2*((precision2 * recall2)/(precision2 + recall2))

print("precision2: {:.4f}, recall2: {:.4f}, F1-measure: {:.4f}".format(precision2, recall2, f1_score2))

precision2: 0.1982, recall2: 0.6792, F1-measure: 0.3068


In [33]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction2[k])[0],"\n")

Movie:  Edge of Darkness
Actual genre:  revenge, neo noir, murder, violence, flashback
Predicted tag:  ['action' 'comedy' 'evil' 'gothic' 'horror' 'murder' 'mystery' 'revenge'
 'suspenseful' 'versus' 'violence'] 

Movie:  Friday the 13th Part III
Actual genre:  cult, horror, murder, violence
Predicted tag:  ['comedy' 'cruelty' 'cult' 'entertaining' 'flashback' 'gothic' 'horror'
 'humor' 'insanity' 'murder' 'noir' 'revenge' 'sadist' 'suspenseful'
 'versus' 'violence'] 

Movie:  Paulie
Actual genre:  entertaining
Predicted tag:  ['melodrama' 'psychedelic'] 

Movie:  Trouble Along the Way
Actual genre:  violence, flashback
Predicted tag:  ['comedy' 'cruelty' 'entertaining' 'melodrama' 'romantic' 'satire'
 'tragedy'] 

Movie:  To Be or Not to Be
Actual genre:  satire
Predicted tag:  ['comedy' 'cruelty' 'flashback' 'humor' 'murder' 'sadist' 'satire'
 'violence'] 



<h2> OneVsRestClassifier + LogisticRegression:</h2>

In [34]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced'))

In [35]:
prediction3 = clf.predict(X_test_multilabel)

precision3 = precision_score(y_test_multilabel, prediction3, average='micro')

recall3 = recall_score(y_test_multilabel, prediction3, average='micro')

f1_score3 = 2*((precision3 * recall3)/(precision3 + recall3))

print("precision3: {:.4f}, recall3: {:.4f}, F1-measure: {:.4f}".format(precision3, recall3, f1_score3))

precision3: 0.2119, recall3: 0.6575, F1-measure: 0.3205


In [36]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction3[k])[0],"\n")

Movie:  The Sign of Four: Sherlock Holmes' Greatest Case
Actual genre:  revenge
Predicted tag:  ['dramatic' 'flashback' 'murder' 'mystery' 'tragedy'] 

Movie:  Niagara, Niagara
Actual genre:  tragedy, violence, murder
Predicted tag:  ['boring' 'humor' 'romantic'] 

Movie:  Artemis Fowl
Actual genre:  fantasy
Predicted tag:  ['action' 'boring' 'cult' 'evil' 'fantasy' 'good' 'psychedelic'
 'suspenseful' 'versus'] 

Movie:  Teenagers from Outer Space
Actual genre:  murder
Predicted tag:  ['action' 'boring' 'comedy' 'cult' 'evil' 'fantasy' 'flashback' 'good'
 'humor' 'murder' 'paranormal' 'psychedelic' 'revenge' 'versus' 'violence'] 

Movie:  Welcome to Me
Actual genre:  comedy, satire
Predicted tag:  ['boring' 'comedy' 'cult' 'dramatic' 'entertaining' 'humor' 'insanity'
 'psychedelic' 'satire'] 



<h1>Conclusion</h1>

In [37]:
from prettytable import PrettyTable

tabel = PrettyTable()

tabel.field_names=['Model', 'Vectorizer', 'Precision','recall','f1_score']


tabel.add_row(['SGDClassifier(log)', 'AVG W2V',  round(precision1, 3), round(recall1, 3), round(f1_score1, 3)])

tabel.add_row(['SGDClassifier(hinge)','AVG W2V', round(precision2, 3),  round(recall2, 3), round(f1_score2, 3)])

tabel.add_row(['LogisticRegression','AVG W2V', round(precision3, 3), round(recall3, 3), round(f1_score3, 3)])


print(tabel)

+----------------------+------------+-----------+--------+----------+
|        Model         | Vectorizer | Precision | recall | f1_score |
+----------------------+------------+-----------+--------+----------+
|  SGDClassifier(log)  |  AVG W2V   |   0.199   | 0.664  |  0.306   |
| SGDClassifier(hinge) |  AVG W2V   |   0.198   | 0.679  |  0.307   |
|  LogisticRegression  |  AVG W2V   |   0.212   | 0.658  |   0.32   |
+----------------------+------------+-----------+--------+----------+
